In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tabular_train_data_path = '../input/petfinder-pawpularity-score/train.csv'
tabular_test_data_path = '../input/petfinder-pawpularity-score/test.csv'
base_path = '../input/petfinder-pawpularity-score'

In [ ]:
train_df = pd.read_csv(tabular_train_data_path)
train_df.describe()

In [ ]:
train_df.head()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df.isna().sum()

In [ ]:
train_df['path'] = base_path + '/train/' + train_df['Id'] + '.jpg'

train_df.head()

In [ ]:
table_input = train_df.iloc[:, 1:-2]
table_input.head(2)

In [ ]:
target = train_df.iloc[:, -2]
target.head(2)

In [ ]:
train_dir = base_path + '/train/'
IMG_SIZE = (256, 256)
train_images = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                   image_size=IMG_SIZE,
                                                                   labels=None,
                                                                   label_mode=None,
                                                                   batch_size=64,
                                                                   shuffle=False)

In [ ]:
AUTO = tf.data.AUTOTUNE  # optimise the pipeline performance

In [ ]:
def preprocess_train(image):
    image = tf.image.random_flip_left_right(image)
    image = image / 255.0
    return image

In [ ]:
train_img_data = train_images.map(preprocess_train, num_parallel_calls=AUTO)
train_img_data = train_img_data.prefetch(AUTO)

In [ ]:
image_batch = next(iter(train_img_data))

plt.figure(figsize=(15, 10))
for n in range(25):
    ax = plt.subplot(5, 5, n + 1)
    plt.imshow(image_batch[n])
    plt.axis("off");

In [ ]:
for image in train_img_data.take(1):
    print("Image Size: ", image.shape)
    print("Image max val: ", tf.reduce_max(image))
    print("Image min val: ", tf.reduce_min(image))
    break

## Building the Model and it's components

In [ ]:
# bit_model_url = 'https://tfhub.dev/google/bit/m-r50x1/1'
# bit_module = hub.KerasLayer(bit_model_url)

# # image model
# img_inputs = tf.keras.layers.Input(shape=(256, 256, 3), name='img_input')
# base_model = bit_module(img_inputs)
# img_outputs = tf.keras.layers.Dense(512, activation='relu')(base_model)
# img_model = tf.keras.Model(img_inputs, img_outputs)

# img_model.summary()

### Model HyperParameters

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 20
image_size = 256  # Final Image Size
patch_size = 32  # Patch Dimension
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers

### 1. Multilayer Perceptron (MLP)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = tf.keras.layers.Dense(units, activation=tf.nn.gelu)(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    return x

### 2. Patch Creation Layer

In [ ]:
class Patches(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
        
    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = 'VALID',
        )
        
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

* Let's see example patched image

In [ ]:
plt.figure(figsize=(10, 8))
for img in train_img_data.take(1):
    image = img[np.random.choice(range(batch_size))]
    plt.imshow(image)
    plt.axis(False)
    
    patches = Patches(patch_size)(tf.expand_dims(image, axis=0))
    print(f"Image size: {image_size} X {image_size}")
    print(f"Patch size: {patch_size} X {patch_size}")
    print(f"Patches per image: {patches.shape[1]}")
    print(f"Elements per patch: {patches.shape[-1]}")
    
    n = int(np.sqrt(patches.shape[1]))
    plt.figure(figsize=(10, 8))
    for i, patch in enumerate(patches[0]):
        ax = plt.subplot(n, n, i+1)
        patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
        plt.imshow(patch_img.numpy())
        plt.axis(False)

### 3. Patch encoder Layer

In [ ]:
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = tf.keras.layers.Dense(projection_dim)
        self.position_embedding = tf.keras.layers.Embedding(input_dim=num_patches, output_dim=projection_dim)
        
    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

### 3. Build the ViT model

In [ ]:
def vit_transformers(inputs):
#     img_inputs = tf.keras.layers.Input(shape=(image_size, image_size, 3))
    
    # create patches.
    patches = Patches(patch_size)(inputs)
    
    # encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    # create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Normalization layer 1.
        x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        # Multi-headed attention layer
        attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        
        # Skip Connection 1.
        x2 = tf.keras.layers.Add()([attention_output, encoded_patches])
        
        # Normalization layer 2.
        x3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # MLP
        x4 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        
        # Skip Connection 2.
        encoded_patches = tf.keras.layers.Add()([x4, x2])
        
    # Create a [batch_size, projection_dim] tensor
    representation = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = tf.keras.layers.Flatten()(representation)
    representation = tf.keras.layers.Dropout(0.5)(representation)
    
    # MLP
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    
    # classify outputs
#     logits = layers.Dense(num_classes)(features)

    model = tf.keras.Model(inputs=inputs, outputs=features)
    return model

In [ ]:
img_inputs = tf.keras.layers.Input(shape=(image_size, image_size, 3))
img_model = vit_transformers(img_inputs)

img_model.summary()

### Tabular Model

In [ ]:
# tabular model
tabular_inputs = tf.keras.layers.Input(shape=(12, ), name='tabular_input')
fc1 = tf.keras.layers.Dense(16, activation='relu')(tabular_inputs) 
fc2 = tf.keras.layers.Dense(32, activation='relu')(fc1)
fc3 = tf.keras.layers.Dense(32, activation='relu')(fc2)
tabular_outputs = tf.keras.layers.Dense(16)(fc3)
tabular_model = tf.keras.Model(tabular_inputs, tabular_outputs)

tabular_model.summary()

In [ ]:
concat_models = tf.keras.layers.Concatenate(axis=1)([img_model.output, tabular_model.output])
output = tf.keras.layers.Dense(1, activation='linear')(concat_models)

model = tf.keras.Model(inputs=[img_inputs, tabular_inputs], outputs=output)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
table_input.shape, target.shape

In [ ]:
table_input = table_input.to_numpy()
target = target.to_numpy()

In [ ]:
table_input[:10], target[:10]

In [ ]:
table_input_data = tf.data.Dataset.from_tensor_slices(table_input)
table_target_data = tf.data.Dataset.from_tensor_slices(target)

In [ ]:
table_ex_data = table_input_data.batch(64).prefetch(AUTO)

In [ ]:
table_target_ex = table_target_data.batch(64).prefetch(AUTO)

In [ ]:
table_target_ex

In [ ]:
train_input_data = tf.data.Dataset.zip((train_img_data, table_ex_data))
train_data = tf.data.Dataset.zip((train_input_data, table_target_ex))

In [ ]:
train_data

### Compile, Train and Evaluate the Model

In [ ]:
model.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay), 
    loss = tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
)

In [ ]:
history = model.fit(train_data,
                   epochs=num_epochs)

In [ ]:
# model.save('./pawpularity.h5')

In [ ]:
def plot_loss_curves(history):
  loss = history.history['loss']
#   val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
#   val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
#   plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
#   plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
loss = history.history['loss']
epochs = range(len(history.history['rmse']))

plt.plot(epochs, loss, label='training_loss')
plt.title('Loss')
plt.xlabel('epochs');

In [ ]:
rmse = history.history['rmse']
plt.plot(epochs, rmse, label='training_rmse')
plt.title('RMSE_loss')
plt.xlabel('epochs');

In [ ]:
test_df = pd.read_csv(tabular_test_data_path)
test_df.head(10)

In [ ]:
test_dir = base_path + '/test/'

In [ ]:
test_input = test_df.iloc[:, 1:]

test_input.head(10)

In [ ]:
test_input = test_input.to_numpy()

In [ ]:
test_input[0]

In [ ]:
img = test_dir + test_df['Id'][0] + '.jpg'

img

In [ ]:
def preprocess_test(image):
    image = tf.io.read_file(image)
    image = tf.io.decode_image(image, channels=3)
    image = tf.image.resize(image, (256, 256))
    image = image / 255.0
    return image

In [ ]:
image0 = preprocess_test(img)
tf.expand_dims(image0, axis=0).shape

In [ ]:
y_pred = model.predict((tf.expand_dims(image0, axis=0), tf.expand_dims(test_input[0], axis=0)))

In [ ]:
y_pred

In [ ]:
y_pred[0].item()


In [ ]:
Pawpularity = []
for i in range(len(test_df)):
    img = test_dir + test_df['Id'][i] + '.jpg'
    image = preprocess_test(img)
    y_pred = model.predict((tf.expand_dims(image, axis=0), tf.expand_dims(test_input[i], axis=0)))
    
    Pawpularity.append(y_pred[0].item())
    
Pawpularity

In [ ]:
sub = {
    'id' : test_df.Id,
    'Pawpularity' : Pawpularity
}

sub_df = pd.DataFrame(sub)
sub_df

In [ ]:
sub_df.to_csv('submission.csv', index=False)